---
title: "Uniform Theory of Diffraction: how to compute the transition function"
categories:
  - Blog
tags:
  - diffraction
  - programming
  - python
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt